In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Layer, Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist

# Function API 생성하기

INPUT_SIZE = 28

def create_model():
  input_tensor = Input(shape = (INPUT_SIZE, INPUT_SIZE))
  x = Flatten()(input_tensor)
  x = Dense(100, activation = 'relu')(x)
  x = Dense(30, activation = 'relu')(x)
  output = Dense(10, activation = 'softmax')(x)

  model = Model(inputs = input_tensor, outputs = output)

  return model

In [4]:
def get_preprocessed_data(images, labels):
  images = np.array(images/255.0, dtype = np.float32)
  labels = np.array(labels, dtype = np.float32)

  return images, labels

def get_preprocessed_ohe(images, labels):
  images, labels = get_preprocessed_data(images, labels)
  oh_labels = to_categorical(labels)

  return images, oh_labels

def get_train_val_test_set(train_images, train_labels, test_images, test_labels, valid_size = 0.15, random_state = 2022):
  train_images, train_oh_labels = get_preprocessed_ohe(train_images, train_labels)
  test_images, test_oh_labels = get_preprocessed_ohe(test_images, test_labels)

  train_images, val_images, train_oh_labels, val_oh_labels = train_test_split(train_images, train_oh_labels, test_size = valid_size, random_state = random_state)
  
  return (train_images, train_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels)


In [10]:
# 모델 형성하기
model = create_model()
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
# 데이터 불러오기
(train_images, train_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels) = get_train_val_test_set(train_images, train_labels, test_images, test_labels, valid_size = 0.15, random_state = 2022)
# 모델 
model.compile(optimizer = Adam(0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])


4431872/4422102 [==============================] - 0s 0us/step


In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

mcp_cb = ModelCheckpoint(filepath = '/content/weights.weights.{epoch:02d}-{val_loss:.2f}.hdf5', moiter = 'val_loss', save_bset_only = True, save_weights_only= True, mode = 'min', period = 1, verbose = 0)
rlp_cb = ReduceLROnPlateau(moniter = 'val_loss', factor = 0.3, patience = 5, mode = 'min', period = 1, verbose = 0)
es_cb = EarlyStopping(monitor='val_loss', patience = 10, mode = 'min', verbose = 1)

history = model.fit(x = train_images, y = train_oh_labels, validation_data= (val_images, val_oh_labels), batch_size = 32, epochs = 40, callbacks = [mcp_cb, rlp_cb, es_cb], verbose = 1)

Epoch 1/40
1594/1594 [==============================] - 8s 4ms/step - loss: 0.5397 - accuracy: 0.8101 - val_loss: 0.4294 - val_accuracy: 0.8461 - lr: 0.0010
Epoch 2/40
1594/1594 [==============================] - 6s 4ms/step - loss: 0.3924 - accuracy: 0.8591 - val_loss: 0.3953 - val_accuracy: 0.8572 - lr: 0.0010
Epoch 3/40
1594/1594 [==============================] - 6s 4ms/step - loss: 0.3512 - accuracy: 0.8725 - val_loss: 0.3538 - val_accuracy: 0.8677 - lr: 0.0010
Epoch 4/40
1594/1594 [==============================] - 7s 5ms/step - loss: 0.3274 - accuracy: 0.8797 - val_loss: 0.3413 - val_accuracy: 0.8781 - lr: 0.0010
Epoch 5/40
1594/1594 [==============================] - 7s 4ms/step - loss: 0.3091 - accuracy: 0.8853 - val_loss: 0.3514 - val_accuracy: 0.8763 - lr: 0.0010
Epoch 6/40
1594/1594 [==============================] - 7s 5ms/step - loss: 0.2941 - accuracy: 0.8908 - val_loss: 0.3283 - val_accuracy: 0.8840 - lr: 0.0010
Epoch 7/40
1594/1594 [==============================] - 7s

In [14]:
model.evaluate(test_images, test_oh_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.3554 - accuracy: 0.8978


[0.3553667664527893, 0.8978000283241272]